# Setting

In [ ]:
import numpy as np
import random 
import os
import sys
import pandas as pd
from tqdm import tqdm

import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [ ]:
import tensorflow as tf 

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Concatenate,AveragePooling1D
from keras.models import Model, load_model

from keras.optimizers import Adam, RMSprop

## From here: 


In [ ]:
TUNING_MODE = 'OFF' # ON/OFF
LOSS_FUNCTION = 'mse'

do_the_mcdropout_wc = True
do_the_mcdropout = True

METRIC = 'MAE'  # MAE/MSE

OPTIMIZER = 'adam' # adam, rmsprop
LR = 0.001   #adam과 rmsprop의 defaut값
BATCH_SIZE = 16
EPOCHS = 600

#-------아직 구현 안 된 부분-------

DECAY_RATE=0.2
DECAY_STEP=200

# Import Data

In [ ]:
model_path = ''
data_path = ''

In [ ]:
data_train = np.load(data_path + 'data_train.npy')
data_train_FGS = np.load(data_path + 'data_train_FGS.npy')

data_test = np.load(data_path + 'data_test.npy')
data_test_FGS = np.load(data_path + 'data_test_FGS.npy')

data_train.shape, data_train_FGS.shape, data_test.shape, data_test_FGS.shape

# Preprocessing

# Model train

### Define

In [ ]:
input_wc = Input((187,1))
x = Conv1D(32, 3, activation='relu')(input_wc)
x = MaxPooling1D()(x)
x = BatchNormalization() (x)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D()(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D()(x)
x = Conv1D(256, 3, activation='relu')(x)
x = MaxPooling1D()(x)
x = Flatten()(x)

x = Dense(500, activation='relu')(x)
x = Dropout(0.2)(x, training = True)
x = Dense(100, activation='relu')(x)
x = Dropout(0.1)(x, training = True)
output_wc = Dense(1, activation='linear')(x)

model_wc = Model(inputs=input_wc, outputs=output_wc)
model_wc.summary()

### Split dataset

### Train Model

### Model Eval

#### Learning curve

#### MSE

# Load Model, Testset

완성한 모델, testset 저장해두기. *이후 `model_concat`에서 부를 예정*